# AI Fashion Assistant v2.4 - System Integration

**Connecting User Management, Personalization & AI Agent**

---

**Project:** AI Fashion Assistant (TÜBİTAK 2209-A)  
**Student:** Hatice Baydemir  
**Date:** January 5, 2026  
**Version:** 2.4.0

---

## Overview

This notebook integrates three major components:
1. User Management System (notebook 01)
2. Personalization Engine (notebook 02)
3. v2.3 AI Agent System

### Integration Architecture

```
User Request → Agent
                ↓
         User Profile → Personalization Engine
                ↓
         Personalized Search Results
                ↓
         RAG-Enhanced Response
```

---

## PART 1: Setup & Imports

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ai_fashion_assistant_v2')

print('Drive mounted')
print(f'Working directory: {os.getcwd()}')

Mounted at /content/drive
Drive mounted
Working directory: /content/drive/MyDrive/ai_fashion_assistant_v2


In [2]:
import json
import sys
from pathlib import Path
from typing import Dict, List, Optional
from datetime import datetime

# Add v2.4 source to path
sys.path.insert(0, 'v2.4-complete/src')

print('Imports complete')

Imports complete


---

## PART 2: Load User Management System

In [3]:
# Import from notebook 01
import threading
from dataclasses import dataclass, field, asdict
from collections import Counter, defaultdict

# Load user management classes (simplified versions)
USER_DATA_DIR = Path('v2.4-complete/data/users')

class UserDataLoader:
    """Load user profiles, history, and favorites"""

    def __init__(self, data_dir: Path):
        self.data_dir = Path(data_dir)

    def load_user(self, user_id: str) -> Dict:
        with open(self.data_dir / 'users.json', 'r') as f:
            users = json.load(f)
        return users.get(user_id, {})

    def load_history(self, user_id: str) -> List[Dict]:
        history_file = self.data_dir / f'history_{user_id}.json'
        if not history_file.exists():
            return []
        with open(history_file, 'r') as f:
            return json.load(f)

    def load_favorites(self, user_id: str) -> Dict[str, Dict]:
        favorites_file = self.data_dir / f'favorites_{user_id}.json'
        if not favorites_file.exists():
            return {}
        with open(favorites_file, 'r') as f:
            return json.load(f)

user_loader = UserDataLoader(USER_DATA_DIR)
print('User management system loaded')

User management system loaded


---

## PART 3: Load Personalization Engine

In [4]:
# Check if personalization results exist
RESULTS_DIR = Path('v2.4-complete/evaluation/results')

if not RESULTS_DIR.exists():
    print('Warning: Personalization results not found')
    print('Please run notebook 02 first')
else:
    # Load pre-computed recommendations
    recommendation_cache = {}

    for user_file in RESULTS_DIR.glob('recommendations_*.csv'):
        user_id = user_file.stem.replace('recommendations_', '')
        import pandas as pd
        recs = pd.read_csv(user_file)
        recommendation_cache[user_id] = recs.to_dict('records')

    print(f'Loaded recommendations for {len(recommendation_cache)} users')

Loaded recommendations for 3 users


---

## PART 4: Enhanced Agent Tools

In [5]:
class PersonalizedSearchTool:
    """Search tool with user personalization"""

    def __init__(self, user_loader: UserDataLoader, recommendation_cache: Dict):
        self.user_loader = user_loader
        self.recommendation_cache = recommendation_cache

    def search(self, user_id: str, query: str, n: int = 5) -> List[Dict]:
        """Personalized search based on user profile"""
        user = self.user_loader.load_user(user_id)

        if not user:
            return []

        # Get user's personalized recommendations
        recommendations = self.recommendation_cache.get(user_id, [])

        # Filter by query keywords
        query_lower = query.lower()
        filtered = []

        for rec in recommendations:
            name = rec.get('product_name', '').lower()
            category = rec.get('category', '').lower()

            # Simple keyword matching
            if any(word in name or word in category for word in query_lower.split()):
                filtered.append(rec)

        return filtered[:n]

    def get_for_you(self, user_id: str, n: int = 10) -> List[Dict]:
        """Get 'For You' recommendations"""
        return self.recommendation_cache.get(user_id, [])[:n]


class UserProfileTool:
    """Tool to access and update user profile"""

    def __init__(self, user_loader: UserDataLoader):
        self.user_loader = user_loader

    def get_preferences(self, user_id: str) -> Dict:
        """Get user preferences"""
        user = self.user_loader.load_user(user_id)
        return user.get('preferences', {})

    def get_favorites(self, user_id: str) -> List[Dict]:
        """Get user's favorite products"""
        favorites = self.user_loader.load_favorites(user_id)
        return list(favorites.values())

    def get_history(self, user_id: str, n: int = 5) -> List[Dict]:
        """Get recent search history"""
        history = self.user_loader.load_history(user_id)
        return history[-n:] if history else []

print('Enhanced agent tools defined')

Enhanced agent tools defined


---

## PART 5: Integrated Agent System

In [6]:
class IntegratedFashionAgent:
    """Agent with user personalization capabilities"""

    def __init__(self, user_loader: UserDataLoader, recommendation_cache: Dict):
        self.search_tool = PersonalizedSearchTool(user_loader, recommendation_cache)
        self.profile_tool = UserProfileTool(user_loader)

    def handle_request(self, user_id: str, query: str) -> Dict:
        """Handle user request with personalization"""
        # Analyze query intent
        query_lower = query.lower()

        response = {
            'user_id': user_id,
            'query': query,
            'timestamp': datetime.now().isoformat(),
            'results': [],
            'personalized': False
        }

        # Intent: Show me recommendations / for you
        if any(word in query_lower for word in ['recommend', 'for me', 'for you', 'suggest']):
            response['results'] = self.search_tool.get_for_you(user_id, n=10)
            response['personalized'] = True
            response['intent'] = 'recommendations'

        # Intent: Show my favorites
        elif any(word in query_lower for word in ['favorite', 'saved', 'liked']):
            response['results'] = self.profile_tool.get_favorites(user_id)
            response['intent'] = 'favorites'

        # Intent: Search with personalization
        else:
            response['results'] = self.search_tool.search(user_id, query, n=10)
            response['personalized'] = True
            response['intent'] = 'search'

        return response

    def generate_response_text(self, response: Dict) -> str:
        """Generate natural language response"""
        user = self.profile_tool.user_loader.load_user(response['user_id'])
        user_name = user.get('name', 'User')

        if not response['results']:
            return f"Sorry {user_name}, I couldn't find any products matching your request."

        # Build response
        lines = []

        if response['intent'] == 'recommendations':
            lines.append(f"Hi {user_name}! Here are my personalized recommendations for you:")
        elif response['intent'] == 'favorites':
            lines.append(f"Hi {user_name}! Here are your saved favorites:")
        else:
            lines.append(f"Hi {user_name}! I found these products for you:")

        lines.append("")

        for i, item in enumerate(response['results'][:5], 1):
            name = item.get('product_name', 'Unknown')
            category = item.get('category', 'product')
            lines.append(f"{i}. {name} ({category})")

        if len(response['results']) > 5:
            lines.append(f"\n... and {len(response['results']) - 5} more items")

        return '\n'.join(lines)

# Initialize agent
agent = IntegratedFashionAgent(user_loader, recommendation_cache)
print('Integrated agent system ready')

Integrated agent system ready


---

## PART 6: Test Integration

In [7]:
# Test scenarios for each user
test_scenarios = [
    ('U001', 'Show me recommendations'),
    ('U001', 'I need a blue dress'),
    ('U002', 'What are my favorites?'),
    ('U002', 'Find me a formal suit'),
    ('U003', 'Recommend something for me'),
    ('U003', 'Show me vintage dresses'),
]

print('Testing Integrated Agent')
print('='*60)

for user_id, query in test_scenarios:
    user = user_loader.load_user(user_id)
    print(f"\n{user['name']} ({user_id}): '{query}'")
    print('-'*60)

    response = agent.handle_request(user_id, query)
    text = agent.generate_response_text(response)

    print(text)
    print(f"\n[Intent: {response['intent']}, Personalized: {response['personalized']}, Results: {len(response['results'])}]")

print('\n' + '='*60)

Testing Integrated Agent

Alice Johnson (U001): 'Show me recommendations'
------------------------------------------------------------
Hi Alice Johnson! Here are my personalized recommendations for you:

1. White Dress (shoes)
2. Navy Jacket (shoes)
3. Blue Jacket (shoes)
4. Yellow Jacket (shoes)
5. Gray Pants (suit)

... and 5 more items

[Intent: recommendations, Personalized: True, Results: 10]

Alice Johnson (U001): 'I need a blue dress'
------------------------------------------------------------
Hi Alice Johnson! I found these products for you:

1. White Dress (shoes)
2. Navy Jacket (shoes)
3. Blue Jacket (shoes)
4. Yellow Jacket (shoes)
5. Gray Pants (suit)

... and 5 more items

[Intent: search, Personalized: True, Results: 10]

Bob Smith (U002): 'What are my favorites?'
------------------------------------------------------------
Hi Bob Smith! Here are your saved favorites:

1. Black Suit (suit)
2. Navy Shirt (shirt)

[Intent: favorites, Personalized: False, Results: 2]

Bob S

---

## PART 7: Conversation Flow Test

In [8]:
def conversation_test(user_id: str, queries: List[str]):
    """Test multi-turn conversation"""
    user = user_loader.load_user(user_id)
    print(f"\nConversation with {user['name']}")
    print('='*60)

    for i, query in enumerate(queries, 1):
        print(f"\nTurn {i}:")
        print(f"User: {query}")

        response = agent.handle_request(user_id, query)
        text = agent.generate_response_text(response)

        print(f"Agent: {text}")

    print('\n' + '='*60)

# Test conversation with Alice
conversation_test('U001', [
    'Hi! What do you recommend for me?',
    'Show me blue dresses',
    'What are my saved favorites?'
    ])

# Test conversation with Bob
conversation_test('U002', [
    'I need formal wear',
    'Show me my favorites'
])


Conversation with Alice Johnson

Turn 1:
User: Hi! What do you recommend for me?
Agent: Hi Alice Johnson! Here are my personalized recommendations for you:

1. White Dress (shoes)
2. Navy Jacket (shoes)
3. Blue Jacket (shoes)
4. Yellow Jacket (shoes)
5. Gray Pants (suit)

... and 5 more items

Turn 2:
User: Show me blue dresses
Agent: Hi Alice Johnson! I found these products for you:

1. Blue Jacket (shoes)
2. Blue Jacket (jacket)
3. Blue Shirt (shoes)
4. Blue Suit (shoes)
5. Blue Shirt (jacket)

Turn 3:
User: What are my saved favorites?
Agent: Hi Alice Johnson! Here are your saved favorites:

1. Blue Summer Dress (dress)
2. White Sneakers (shoes)
3. Gray Jacket (jacket)


Conversation with Bob Smith

Turn 1:
User: I need formal wear
Agent: Hi Bob Smith! I found these products for you:

1. Navy Jacket (suit)
2. Navy Accessories (jacket)
3. White Pants (shirt)
4. Orange Shirt (suit)
5. Red Suit (suit)

... and 5 more items

Turn 2:
User: Show me my favorites
Agent: Hi Bob Smith! Here 

---

## PART 8: Performance Metrics

In [9]:
import time

def benchmark_agent(n_queries: int = 50):
    """Benchmark agent performance"""
    users = ['U001', 'U002', 'U003']
    queries = [
        'Show me recommendations',
        'Find me a dress',
        'What are my favorites?',
    ]

    response_times = []

    for _ in range(n_queries):
        user_id = users[_ % len(users)]
        query = queries[_ % len(queries)]

        start = time.time()
        response = agent.handle_request(user_id, query)
        end = time.time()

        response_times.append(end - start)

    import numpy as np
    return {
        'total_queries': n_queries,
        'avg_response_time': np.mean(response_times),
        'min_response_time': np.min(response_times),
        'max_response_time': np.max(response_times),
        'p95_response_time': np.percentile(response_times, 95)
    }

print('Benchmarking agent performance...')
metrics = benchmark_agent(n_queries=50)

print('\nPerformance Metrics:')
print('='*60)
for key, value in metrics.items():
    if 'time' in key:
        print(f'  {key}: {value*1000:.2f} ms')
    else:
        print(f'  {key}: {value}')
print('='*60)

Benchmarking agent performance...

Performance Metrics:
  total_queries: 50
  avg_response_time: 11.92 ms
  min_response_time: 0.03 ms
  max_response_time: 496.49 ms
  p95_response_time: 4.07 ms


---

## PART 9: Integration Analysis

In [10]:
def analyze_integration():
    """Analyze integration effectiveness"""
    users = ['U001', 'U002', 'U003']

    stats = {
        'users': len(users),
        'personalization_available': len(recommendation_cache),
        'avg_recommendations_per_user': 0,
        'user_profiles': {},
    }

    total_recs = 0

    for user_id in users:
        user = user_loader.load_user(user_id)
        favorites = user_loader.load_favorites(user_id)
        history = user_loader.load_history(user_id)
        recs = recommendation_cache.get(user_id, [])

        total_recs += len(recs)

        stats['user_profiles'][user_id] = {
            'name': user.get('name'),
            'preferences': user.get('preferences', {}),
            'favorites_count': len(favorites),
            'search_history_count': len(history),
            'recommendations_count': len(recs)
        }

    stats['avg_recommendations_per_user'] = total_recs / len(users)

    return stats

print('Integration Analysis')
print('='*60)

analysis = analyze_integration()

print(f"\nSystem Overview:")
print(f"  Total users: {analysis['users']}")
print(f"  Personalization coverage: {analysis['personalization_available']}/{analysis['users']}")
print(f"  Avg recommendations/user: {analysis['avg_recommendations_per_user']:.1f}")

print(f"\nUser Details:")
for user_id, profile in analysis['user_profiles'].items():
    print(f"\n  {profile['name']} ({user_id}):")
    print(f"    Favorites: {profile['favorites_count']}")
    print(f"    Search history: {profile['search_history_count']}")
    print(f"    Recommendations ready: {profile['recommendations_count']}")
    prefs = profile['preferences']
    if prefs:
        print(f"    Style: {prefs.get('style', [])}")
        print(f"    Colors: {prefs.get('colors', [])}")

print('\n' + '='*60)

Integration Analysis

System Overview:
  Total users: 3
  Personalization coverage: 3/3
  Avg recommendations/user: 20.0

User Details:

  Alice Johnson (U001):
    Favorites: 3
    Search history: 3
    Recommendations ready: 20
    Style: ['casual', 'sporty']
    Colors: ['blue', 'white', 'gray']

  Bob Smith (U002):
    Favorites: 2
    Search history: 2
    Recommendations ready: 20
    Style: ['formal', 'elegant']
    Colors: ['black', 'navy', 'gray']

  Carol Williams (U003):
    Favorites: 2
    Search history: 2
    Recommendations ready: 20
    Style: ['vintage']
    Colors: ['red', 'orange', 'yellow']



---

## PART 10: Save Integration Results

In [11]:
OUTPUT_DIR = Path('v2.4-complete/evaluation/results')
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Save integration analysis
analysis_file = OUTPUT_DIR / 'integration_analysis.json'
with open(analysis_file, 'w') as f:
    json.dump(analysis, f, indent=2)
print(f'Saved: {analysis_file}')

# Save performance metrics
metrics_file = OUTPUT_DIR / 'integration_performance.json'
with open(metrics_file, 'w') as f:
    json.dump(metrics, f, indent=2)
print(f'Saved: {metrics_file}')

# Save test results
test_results = []
for user_id, query in test_scenarios:
    response = agent.handle_request(user_id, query)
    test_results.append({
        'user_id': user_id,
        'query': query,
        'intent': response['intent'],
        'personalized': response['personalized'],
        'results_count': len(response['results'])
    })

test_file = OUTPUT_DIR / 'integration_test_results.json'
with open(test_file, 'w') as f:
    json.dump(test_results, f, indent=2)
print(f'Saved: {test_file}')

print('\nAll integration results saved')

Saved: v2.4-complete/evaluation/results/integration_analysis.json
Saved: v2.4-complete/evaluation/results/integration_performance.json
Saved: v2.4-complete/evaluation/results/integration_test_results.json

All integration results saved


---

## Summary

In [12]:
print('='*60)
print('SYSTEM INTEGRATION COMPLETE')
print('='*60)

print('\nIntegrated Components:')
print('  1. User Management System')
print('  2. Personalization Engine')
print('  3. AI Agent System')

print('\nKey Features:')
print('  - Personalized search results')
print('  - User preference awareness')
print('  - Favorite product access')
print('  - Search history integration')
print('  - Intent recognition')

print('\nPerformance:')
print(f'  - Avg response time: {metrics["avg_response_time"]*1000:.2f} ms')
print(f'  - P95 response time: {metrics["p95_response_time"]*1000:.2f} ms')

print('\nCoverage:')
print(f'  - Users: {analysis["users"]}')
print(f'  - Recommendations ready: {analysis["personalization_available"]}')
print(f'  - Avg recs/user: {analysis["avg_recommendations_per_user"]:.1f}')

print('\nOutput Files:')
print(f'  - {analysis_file}')
print(f'  - {metrics_file}')
print(f'  - {test_file}')

print('\nNext Steps:')
print('  - Deploy as API endpoint')
print('  - Add conversational memory')
print('  - Implement feedback loop')

print('='*60)

SYSTEM INTEGRATION COMPLETE

Integrated Components:
  1. User Management System
  2. Personalization Engine
  3. AI Agent System

Key Features:
  - Personalized search results
  - User preference awareness
  - Favorite product access
  - Search history integration
  - Intent recognition

Performance:
  - Avg response time: 11.92 ms
  - P95 response time: 4.07 ms

Coverage:
  - Users: 3
  - Recommendations ready: 3
  - Avg recs/user: 20.0

Output Files:
  - v2.4-complete/evaluation/results/integration_analysis.json
  - v2.4-complete/evaluation/results/integration_performance.json
  - v2.4-complete/evaluation/results/integration_test_results.json

Next Steps:
  - Deploy as API endpoint
  - Add conversational memory
  - Implement feedback loop
